In [111]:
import socket
from struct import unpack, pack

In [112]:
class Codec:
    @staticmethod
    def registerRequest_encode(nick, ip, port):
        return pack(f'=BB{len(nick)}s{4}sH', 1, len(nick), nick.encode('utf-8'), socket.inet_aton(ip), port)
    @staticmethod
    def registerResponse_encode(nick, ip, port):
        return pack(f'=BB{len(nick)}s{4}sH', 2, len(nick), nick.encode('utf-8'), socket.inet_aton(ip), port)
    @staticmethod
    def addUser_encode(nick, ip, port):
        return pack(f'=BB{len(nick)}s{4}sH', 3, len(nick), nick.encode('utf-8'), socket.inet_aton(ip), port)
    @staticmethod
    def broadcastMsg_encode(msg):
        return pack(f'=BB{len(msg)}s', 5, len(msg), msg.encode('utf-8'))
    @staticmethod
    def broadcast_encode(nick, msg):
        return pack(f'=BB{len(nick)}sB{len(msg)}s', 6, len(nick), nick.encode('utf-8'), len(msg), msg.encode('utf-8'))
    @staticmethod
    def logout_encode():
        return pack(f'=B', 7)
    @staticmethod
    def removeUser_encode(nick):
        return pack(f'=BB{len(nick)}s', 8, len(nick), nick.encode('utf-8'))
    @staticmethod
    def registerRequest_decode(data):
        decoded = unpack(f'=BB{len(data)-8}sBBBBH', data)
        nick = str(decoded[2].decode('utf-8'))
        ip = str(decoded[3])+'.'+str(decoded[4])+'.'+str(decoded[5])+'.'+str(decoded[6])
        port = decoded[7]
        return (nick, ip, port)
    @staticmethod
    def registerResponse_decode(data):
        decoded = unpack(f'=BB{len(data)-8}sBBBBH', data)
        nick = decoded[2].decode('utf-8')
        ip = str(decoded[3])+'.'+str(decoded[4])+'.'+str(decoded[5])+'.'+str(decoded[6])
        port = decoded[7]
        return (nick, ip, port)
    @staticmethod
    def addUser_decode(data):
        decoded = unpack(f'=BB{len(data)-8}sBBBBH', data)
        nick = decoded[2].decode('utf-8')
        ip = str(decoded[3])+'.'+str(decoded[4])+'.'+str(decoded[5])+'.'+str(decoded[6])
        port = decoded[7]
        return (nick, ip, port)
    @staticmethod
    def broadcastMsg_decode(data):
        decoded = unpack(f'=BB{len(data)-2}s', data)
        return decoded[2].decode('utf-8')
    @staticmethod
    def broadcast_decode(data):
        nicklen = data[1]
        msglen = data[nicklen+2]
        decoded = unpack(f'=BB{nicklen}sB{msglen}s', data)
        nick = decoded[2].decode('utf-8')
        msg = decoded[4].decode('utf-8')
        return (nick, msg)
    @staticmethod
    def removeUser_decode(data):
        nicklen = data[1]
        decoded = unpack(f'=BB{nicklen}s', data)
        return decoded[2].decode('utf-8')

In [113]:
def registerRequest_encode(nick, ip, port):
    return pack(f'BB{len(nick)}s{4}sH', 1, len(nick), nick.encode('utf-8'), socket.inet_aton(ip), port)

def addUser_encode(nick, ip, port):
    return pack(f'BB{len(nick)}s{4}sH', 3, len(nick), nick.encode('utf-8'), socket.inet_aton(ip), port)

def addUser_decode(data):
    return unpack(f'BB{len(data)-9}sBBBBH', data)

def broadcastMsg_encode(msg):
    return pack(f'BB{len(msg)}s', 5, len(msg), msg.encode('utf-8'))

def broadcast_encode(nick, msg):
    return pack(f'BB{len(nick)}sB{len(msg)}s', 6, len(nick), nick.encode('utf-8'), len(msg), msg.encode('utf-8'))

def logout_encode():
    return pack(f'B', 7)

def removeUser_encode(nick):
    return pack(f'BB{len(nick)}s', 8, len(nick), nick.encode('utf-8'))

In [114]:
def addUser_decode2(data):
        decoded = unpack(f'BB{len(data)-9}sBBBBH', data)
        msgtype = decoded[0]
        nick = decoded[2].decode('utf-8')
        ip = str(decoded[3])+'.'+str(decoded[4])+'.'+str(decoded[5])+'.'+str(decoded[6])
        port = decoded[7]
        return (msgtype, nick, ip, port)

In [115]:
def broadcastMsg_decode(data):
        decoded = unpack(f'BB{len(data)-2}s', data)
        return decoded[2].decode('utf-8')

In [116]:
def broadcast_decode(data):
        nicklen = data[1]
        msglen = data[nicklen+2]
        decoded = unpack(f'BB{nicklen}sB{msglen}s', data)
        nick = decoded[2].decode('utf-8')
        msg = decoded[4].decode('utf-8')
        return (nick, msg)

In [117]:
ip = '127.0.0.2'
nick = 'max'
port = 50
msg = 'hallo welt'

In [118]:
encoded = broadcast_encode(nick, msg)
decoded = broadcast_decode(encoded)
nicklen = encoded[1]
msglen = encoded[nicklen+2]
print(encoded)
print(nicklen)
print(msglen)
#print(decoded)

b'\x06\x03max\nhallo welt'
3
10


In [119]:
encoded = broadcastMsg_encode(msg)
decoded = broadcastMsg_decode(encoded)
print(decoded)

hallo welt


In [120]:
userEncoded = addUser_encode(nick, ip, port)
userDecoded = addUser_decode2(userEncoded)
print(userDecoded)

(3, 'max', '127.0.0.2', 50)


In [121]:
print(socket.inet_aton(ip))
print(addUser_encode('max', '127.0.0.2', 50))
print(addUser_decode(addUser_encode('max', '127.0.0.2', 50)))

b'\x7f\x00\x00\x02'
b'\x03\x03max\x7f\x00\x00\x02\x002\x00'
(3, 3, b'max', 127, 0, 0, 2, 50)


In [122]:
data = pack(f'B{4}s', 1, '1234'.encode('utf-8'))
unpacked = unpack(f'B{4}s', data)
print(data[0])
print(unpacked[0])
print(unpacked[1].decode('utf-8'))

1
1
1234


In [123]:
user = dict()
user.update({nick : (ip, port)})
for u in user:
    print(u)
    print(user.get(u))

max
('127.0.0.2', 50)


In [124]:
a = Codec.registerRequest_encode('a', ip, port)
b = Codec.registerRequest_encode('bb', ip, port)
c = Codec.registerRequest_encode('ccc', ip, port)
d = Codec.registerRequest_encode('dddd', ip, port)
e = Codec.registerRequest_encode('eeeee', ip, port)
empty = Codec.registerRequest_encode('', ip, port)
da = Codec.registerRequest_decode(a)
db = Codec.registerRequest_decode(b)
print(len(a))
print(len(b))
print(len(c))
print(len(d))
print(len(e))
print(len(empty))
print(da)
print(db)
print(da[0])
print(db[0])

9
10
11
12
13
8
('a', '127.0.0.2', 50)
('bb', '127.0.0.2', 50)
a
bb


In [125]:
print(len(nick))
print(len(nick.encode('utf-8')))

3
3
